#### If bokeh is not installed, run the next cell:

In [2]:
!pip install bokeh

You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np
import math
from bokeh.plotting import figure, output_notebook, show, output_file
import matplotlib.cm as cm
import matplotlib as mpl
from bokeh.plotting import hplot, vplot
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, PreviewSaveTool
)

In [4]:
# Returns array with all elements between [0,1]
def normalize_array(array):
    array = np.array(array)
    array = array + abs(array.min())
    array = array * 1.0 / array.max()
    return array

In [5]:
def hsv2rgb(h, s, v):
    h = float(h)
    s = float(s)
    v = float(v)
    h60 = h / 60.0
    h60f = math.floor(h60)
    hi = int(h60f) % 6
    f = h60 - h60f
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    r, g, b = 0, 0, 0
    if hi == 0: r, g, b = v, t, p
    elif hi == 1: r, g, b = q, v, p
    elif hi == 2: r, g, b = p, v, t
    elif hi == 3: r, g, b = p, q, v
    elif hi == 4: r, g, b = t, p, v
    elif hi == 5: r, g, b = v, p, q
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return "#%02x%02x%02x" % (r,g,b)

In [6]:
# Takes the array vectors for each station and returns the corresponding RGB colors values for all stations
def get_colors(dim1, dim2, dim3):
    H = normalize_array(dim1) * 360 # varies from 0 to 360
    S = normalize_array(dim2) # varies from 0 to 1. Purity of color
    V = normalize_array(dim2) # varies from 0 to 1. Brightness of color
    colors = []
    for (h,s,v) in zip(H,S,V):
        colors.append(hsv2rgb(h,s,v))
    return colors

In [8]:
import pickle
Data_dir='../../Data/Weather'
stations=pickle.load(open(Data_dir+'/stations.pkl','r'))
stations.head()

,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


### Change 'elevation' to the field names of the 3 variables that need to be plotted

In [9]:
# Get the 3 variables to display in map for each station
variable1=stations.ix[:,'elevation'].values
variable2=stations.ix[:,'elevation'].values
variable3=stations.ix[:,'elevation'].values

In [10]:
#this code is using pandas dataframes
lons=stations.ix[:,'longitude'].values
lats=stations.ix[:,'latitude'].values

ll=len(lons)
lonmin=-180;lonmax=180;latsmin=-80;latsmax=80;
select=(lons>lonmin) * (lons<lonmax)*(lats>latsmin)*(lats<latsmax)
lons=lons[select]
lats=lats[select]
variable1=variable1[select]
variable2=variable2[select]
variable3=variable3[select]

### Sampling from dataset

In [11]:
arr = np.array(zip(lons, lats, variable1, variable2, variable3))
indices = np.random.choice(xrange(len(arr)), 4000, replace=False) # Keeping 4000 points only
new_arr = arr[indices]
lons = new_arr[:,0]
lats = new_arr[:,1]
variable1 = new_arr[:,2]
variable2 = new_arr[:,3]
variable3 = new_arr[:,4]

### Find the color point in HSV cylinder and convert to RGB

In [12]:
colors = get_colors(variable1, variable2, variable3)

### Create output HTML

In [13]:
output_file("plot.html", title="USA Elevation plot", mode="cdn")

Slow 

In [15]:
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, PreviewSaveTool
)

map_options = GMapOptions(lat=37.09024, lng=-95.712891, map_type="terrain", zoom=3)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options)


source = ColumnDataSource(
    data=dict(
        lat=lats,
        lon=lons,
        col=colors
    )
)


circle = Circle(x="lon", y="lat", size=5, fill_color="col", fill_alpha=1, line_color=None,)
plot.add_glyph(source, circle) # Plot for variable 1
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), PreviewSaveTool())

show(plot)


KeyboardInterrupt: 